In [4]:

# Required columns (DB schema)
required_columns = [
    "exchange",
    "exchange_token",
    "trading_symbol",
    "name",
    "instrument_type",
    "segment",
    "series",
    "isin",
    "expiry_date",
    "strike_price",
    "lot_size"
]

# Keep only existing columns
existing_columns = [c for c in required_columns if c in df.columns]

# Filter dataframe
final_df = df.loc[:, existing_columns].copy()
final_df    

NameError: name 'df' is not defined

In [6]:
import duckdb

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\data\symbols\symbols.duckdb"

# CONNECT
con = duckdb.connect(db_path)

# CHECK TABLES
tables = con.execute("SHOW TABLES").fetchall()
print("Tables:", tables)

# ROW COUNT (change table name if needed)
row_count = con.execute("SELECT COUNT(*) FROM symbols").fetchone()[0]
print("Row count:", row_count)

# CLOSE
con.close()


Tables: [('schedulers',), ('series_lookup',), ('series_lookup_metadata',), ('symbols',), ('transformation_scripts',), ('upload_logs',)]
Row count: 172734


In [ ]:
import duckdb

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\data\symbols\symbols.duckdb"

# CONNECT
con = duckdb.connect(db_path)

# GET COLUMN HEADERS
columns = con.execute("DESCRIBE symbols").fetchall()

print("Column headers:")
for col in columns:
    print(col[0])

# CLOSE
con.close()


Column headers:
id
exchange
trading_symbol
exchange_token
name
instrument_type
segment
series
isin
expiry_date
strike_price
lot_size
status
source
created_at
updated_at
last_updated_at


: 

In [3]:
for table in ["symbol_upload_logs", "symbols"]:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count}")


ConnectionException: Connection Error: Connection already closed!

In [ ]:
import json
import os
import pandas as pd
from pathlib import Path
import duckdb
import sqlite3

# Path to connections data
base_path = Path(r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics")
connections_file = base_path / "data" / "connections" / "connections.json"
active_file = base_path / "data" / "connections" / "active_connection.json"

# Load connections
connections_data = {}
active_connections = {}

if connections_file.exists():
    with open(connections_file, 'r') as f:
        data = json.load(f)
        connections_data = {conn["id"]: conn for conn in data.get("connections", [])}

if active_file.exists():
    with open(active_file, 'r') as f:
        active_connections = json.load(f)

# Test connection status
def test_connection(conn_type, config):
    """Test if a database connection is working"""
    try:
        if conn_type == "sqlite" or conn_type == "sqlite3":
            path = config.get("path") or config.get("database")
            if path and os.path.exists(path):
                conn = sqlite3.connect(path)
                conn.execute("SELECT 1")
                conn.close()
                return "✓ Connected"
            return "✗ File not found"
        elif conn_type in ["duckdb", "duckdb_direct", "duckdb_sqlalchemy"]:
            path = config.get("path")
            if path:
                if os.path.exists(path):
                    con = duckdb.connect(path)
                    con.execute("SELECT 1")
                    con.close()
                    return "✓ Connected"
                return "✗ File not found"
            # Multi-database case
            return "✓ Configured"
        else:
            return "? Unknown type"
    except Exception as e:
        return f"✗ Error: {str(e)[:30]}"

# Display connections
print("=" * 100)
print("DATABASE CONNECTIONS STATUS")
print("=" * 100)
print()

if connections_data:
    # Create a list for display
    conn_list = []
    for conn_id, conn in connections_data.items():
        category = conn.get("category", "N/A")
        is_active = conn_id in active_connections.values()
        active_marker = "✓ ACTIVE" if is_active else "  "
        
        conn_type = conn.get("type", "N/A")
        name = conn.get("name", "N/A")
        
        # Get database path(s)
        config = conn.get("config", {})
        if "path" in config:
            db_path = config["path"]
            # Test connection
            conn_status = test_connection(conn_type, config)
        elif conn_type == "duckdb" and any(k in config for k in ["ohlcv", "indicators", "signals", "jobs"]):
            db_path = f"Multiple: {', '.join([k for k in config.keys() if k != 'path'])}"
            conn_status = "✓ Configured (multi-DB)"
        else:
            db_path = "N/A"
            conn_status = "? Not configured"
        
        conn_list.append({
            "Active": active_marker,
            "Name": name,
            "Type": conn_type,
            "Category": category,
            "Connection Status": conn_status,
            "Database Path": str(db_path)[:60] + "..." if len(str(db_path)) > 60 else str(db_path)
        })
    
    # Display as DataFrame
    df_connections = pd.DataFrame(conn_list)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 70)
    print(df_connections.to_string(index=False))
    print()
    
    # Show active connections summary
    print("-" * 100)
    print("ACTIVE CONNECTIONS BY CATEGORY:")
    print("-" * 100)
    for category, conn_id in sorted(active_connections.items()):
        if conn_id:
            conn = connections_data.get(conn_id, {})
            config = conn.get("config", {})
            conn_type = conn.get("type", "N/A")
            status = test_connection(conn_type, config)
            print(f"  {category:20s} → {conn.get('name', conn_id):40s} ({conn_type:15s}) {status}")
        else:
            print(f"  {category:20s} → None")
    print()
    
    # Summary
    active_count = sum(1 for cid in connections_data.keys() if cid in active_connections.values())
    total_count = len(connections_data)
    print(f"Summary: {active_count} active out of {total_count} total connections")
else:
    print("No connections found.")

print()
print("=" * 100)


In [3]:
import yfinance as yf

df = yf.download(
    "RELIANCE.NS",
    start="2025-12-10",
    end="2025-12-12",
    interval="1d"
)

print(df.head())


C:\Users\jallu\AppData\Local\Temp\ipykernel_70964\2181961351.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed

Price             Close        High          Low         Open      Volume
Ticker      RELIANCE.NS RELIANCE.NS  RELIANCE.NS  RELIANCE.NS RELIANCE.NS
Date                                                                     
2025-12-10  1536.900024      1547.5  1531.400024  1534.000000     7991629
2025-12-11  1545.000000      1550.0  1524.000000  1536.900024     4706197


In [36]:
import sqlite3

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\backend\data\auth\sqlite\auth.db"

# CONNECT
con = sqlite3.connect(db_path)
cur = con.cursor()

# CHECK TABLES
cur.execute("""
    SELECT name 
    FROM sqlite_master 
    WHERE type='table';
""")
tables = cur.fetchall()
print("Tables:", tables)

# ROW COUNT (change table name if needed)
table_name = "users"  # <-- replace with actual table name
cur.execute(f"SELECT COUNT(*) FROM {table_name}")
row_count = cur.fetchone()[0]
print("Row count:", row_count)

# CLOSE
con.close()


Tables: [('users',), ('access_requests',), ('feedback',), ('sessions',), ('feature_requests',), ('audit_logs',), ('sqlite_sequence',), ('symbols',), ('transformation_scripts',), ('symbol_upload_logs',), ('scheduled_ingestions',), ('ingestion_sources',), ('connections',)]
Row count: 3


In [ ]:
import sqlite3
import pandas as pd

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\backend\data\auth\sqlite\auth.db"

def fetch_table_safe_df(table_name, limit=5):
    con = sqlite3.connect(db_path)
    cur = con.cursor()

    # VALIDATE TABLE NAME
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    allowed = [t[0] for t in cur.fetchall()]

    if table_name not in allowed:
        con.close()
        raise ValueError("Invalid table name")

    # FETCH FIRST N ROWS
    query = f"""
        SELECT *
        FROM {table_name}
        ORDER BY rowid ASC
        LIMIT {limit}
    """

    df = pd.read_sql_query(query, con)

    con.close()
    return df


# =========================
# USAGE
# =========================
df = fetch_table_safe_df("users", limit=5)

print(df)


   id                username                   email        mobile  \
0   2  testuser@rubikview.com  testuser@rubikview.com    9515625875   
1   4                 sandeep       sandeep@rubik.com    8686504620   
2   5                   admin     admin@rubikview.com  +10000000000   

                                     hashed_password         role  is_active  \
0  $2b$12$u7Sm8k/Rw6dstyT0P4lTnu9.gNJmE6lsZb08Ql9...        admin          1   
1  $2b$12$FXa/jswKk5f1fmGg/hYNC.YAMz/Y8kdMpIzEoaa...  super_admin          1   
2  $2b$12$CoWtm2TUU.8RhkxWsFhct.ISQbNrVflwPRhprmB...  super_admin          1   

            created_at                  updated_at  \
0  2025-12-19 16:29:34  2025-12-23 19:47:00.676215   
1  2025-12-20 12:24:46         2025-12-26 06:23:39   
2  2025-12-21 06:56:55         2025-12-22 06:07:32   

                    last_seen theme_preference           name  \
0  2025-12-20 08:28:11.304847             dark         testin   
1  2025-12-26 06:15:38.376916             dark 

: 

In [1]:
!pip install requests beautifulsoup4 pandas lxml


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://www.screener.in/company/RELIANCE/"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers, timeout=15)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

print("Page title:", soup.title.text.strip())


Page title: Reliance Industries Ltd share price | About Reliance Industr | Key Insights - Screener


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

SYMBOL = "AAA Technologies"
URL = f"https://www.screener.in/company/{SYMBOL}/consolidated/"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/131.0 Safari/537.36"
    ),
    "Referer": "https://www.screener.in/"
}

def fetch_soup(url: str) -> BeautifulSoup:
    resp = requests.get(url, headers=HEADERS, timeout=20)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")

def parse_header_fundamentals(soup: BeautifulSoup) -> dict:
    """
    Extract Market Cap, Price, PE, ROE, etc. from the top info block.
    This uses regex on full text to be more robust to minor HTML changes.
    """
    txt = soup.get_text("\n")

    def grab(pattern):
        m = re.search(pattern, txt)
        return m.group(1).strip() if m else None

    data = {}
    data["Market Cap (Cr)"] = grab(r"Market Cap\s*₹\s*([0-9,\.]+)\s*Cr")
    data["Current Price"] = grab(r"Current Price\s*₹\s*([0-9,\.]+)")
    data["High / Low"] = grab(r"High\s*/\s*Low\s*₹\s*([0-9,\. /]+)")
    data["Stock P/E"] = grab(r"Stock P/E\s*([0-9\.]+)")
    data["Book Value"] = grab(r"Book Value\s*₹\s*([0-9,\.]+)")
    data["Dividend Yield %"] = grab(r"Dividend Yield\s*([0-9\.]+)\s*%")
    data["ROCE %"] = grab(r"ROCE\s*([0-9\.]+)\s*%")
    data["ROE %"] = grab(r"ROE\s*([0-9\.]+)\s*%")
    data["Face Value"] = grab(r"Face Value\s*₹\s*([0-9,\.]+)")
    return data

def parse_peer_table(soup: BeautifulSoup) -> pd.DataFrame | None:
    """
    Find the peer comparison table by looking for columns like 'CMP Rs.' and 'P/E'.
    """
    tables = pd.read_html(str(soup))
    for df in tables:
        cols = [str(c) for c in df.columns]
        joined = " ".join(cols)
        if "CMP Rs." in joined and "P/E" in joined:
            return df
    return None

def parse_section_table(soup: BeautifulSoup, heading_text: str) -> pd.DataFrame | None:
    """
    For blocks like 'Profit & Loss', 'Balance Sheet', 'Cash Flows', 'Ratios'.
    """
    h = soup.find(lambda tag: tag.name in ["h2", "h3"] and heading_text in tag.get_text())
    if not h:
        return None
    table = h.find_next("table")
    if not table:
        return None
    return pd.read_html(str(table))[0]

def main():
    soup = fetch_soup(URL)

    # 1) Header fundamentals
    header_data = parse_header_fundamentals(soup)
    header_df = pd.DataFrame([header_data])
    header_df.to_csv(f"{SYMBOL}_header_fundamentals.csv", index=False)
    print("Header fundamentals:")
    print(header_df.T)

    # 2) Peer comparison
    peer_df = parse_peer_table(soup)
    if peer_df is not None:
        peer_df.to_csv(f"{SYMBOL}_peers.csv", index=False)
        print("\nPeer comparison (top 5):")
        print(peer_df.head())

    # 3) Financial statements
    pl_df = parse_section_table(soup, "Profit & Loss")
    bs_df = parse_section_table(soup, "Balance Sheet")
    cf_df = parse_section_table(soup, "Cash Flows")
    ratio_df = parse_section_table(soup, "Ratios")

    if pl_df is not None:
        pl_df.to_csv(f"{SYMBOL}_profit_loss.csv", index=False)
    if bs_df is not None:
        bs_df.to_csv(f"{SYMBOL}_balance_sheet.csv", index=False)
    if cf_df is not None:
        cf_df.to_csv(f"{SYMBOL}_cash_flows.csv", index=False)
    if ratio_df is not None:
        ratio_df.to_csv(f"{SYMBOL}_ratios.csv", index=False)

    print("\nSaved CSVs for:", SYMBOL)

if __name__ == "__main__":
    main()


IndentationError: expected an indented block after 'if' statement on line 93 (3177863400.py, line 95)

In [17]:
import requests
from bs4 import BeautifulSoup

url = "https://www.screener.in/company/RELIANCE/"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.screener.in/"
}

response = requests.get(url, headers=headers, timeout=15)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

print("Page title:", soup.title.get_text(strip=True))


Page title: Reliance Industries Ltd share price | About Reliance Industr | Key Insights - Screener


In [16]:
import pandas as pd
from bs4 import BeautifulSoup

# 1. Get all tables
tables = pd.read_html(str(soup))

# 2. Find all table elements in HTML (same order as read_html)
html_tables = soup.find_all("table")

print(f"Total tables found: {len(html_tables)}")

# 3. For each table, find nearest preceding heading
for idx, table in enumerate(html_tables):
    heading = None

    # Look backwards for the nearest h2 or h3
    for prev in table.find_all_previous(["h2", "h3"], limit=1):
        heading = prev.get_text(strip=True)
        break

    print("\n==============================")
    print(f"TABLE INDEX: {idx}")
    print("HEADING:", heading if heading else "No heading found")
    print("SHAPE:", tables[idx].shape)
    print(tables[idx].head(5))


Total tables found: 11

TABLE INDEX: 0
HEADING: Quarterly Results
SHAPE: (12, 14)
         Unnamed: 0 Sep 2022 Dec 2022 Mar 2023 Jun 2023 Sep 2023 Dec 2023  \
0           Sales +   137346   125849   129674   122627   137380   127695   
1        Expenses +   125561   110950   110542   105134   118189   110137   
2  Operating Profit    11785    14899    19132    17493    19191    17558   
3             OPM %       9%      12%      15%      14%      14%      14%   
4    Other Income +     3499     2689     2750     2728     2934     2969   

  Mar 2024 Jun 2024 Sep 2024 Dec 2024 Mar 2025 Jun 2025 Sep 2025  
0   146832   129898   130108   124381   132962   116341   126335  
1   126809   115583   116683   109168   117846   103171   111946  
2    20023    14315    13425    15213    15116    13170    14389  
3      14%      11%      10%      12%      11%      11%      11%  
4     3497     3502     3801     3214     5577    13460     3445  

TABLE INDEX: 1
HEADING: Profit & Loss
SHAPE: (12, 14

C:\Users\jallu\AppData\Local\Temp\ipykernel_90600\1036851876.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup))


In [19]:
# Collect headings with possible repeats
raw_headings = []

html_tables = soup.find_all("table")

for table in html_tables:
    heading = None
    for prev in table.find_all_previous(["h2", "h3"], limit=1):
        heading = prev.get_text(strip=True)
        break
    raw_headings.append(heading if heading else "No heading")

# Deduplicate while preserving order
unique_headings = []
for h in raw_headings:
    if h not in unique_headings:
        unique_headings.append(h)

# Print only unique names
for i, h in enumerate(unique_headings):
    print(f"{i}: {h}")


0: Quarterly Results
1: Profit & Loss
2: Balance Sheet
3: Cash Flows
4: Ratios
5: Shareholding Pattern


In [ ]:
import time
import csv
import requests

BASE = "https://www.nseindia.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    ),
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.nseindia.com/companies-listing/corporate-filings-announcements",
}

session = requests.Session()
session.headers.update(HEADERS)

def nse_get(path, params=None, delay=1.5):
    time.sleep(delay)
    url = BASE + path
    r = session.get(url, params=params, timeout=20)
    r.raise_for_status()
    return r.json()

def fetch_nse_announcements(max_records=500, out_csv="nse_announcements.csv"):
    """
    Fetch latest corporate announcements and save to CSV.
    Handles both list and dict-with-data responses.
    """
    data = nse_get(
        "/api/corporate-announcements",
        params={"index": "equities", "from_date": "", "to_date": "", "company": "", "industry": ""},
    )

    # Normalise: API may return list OR {"data": [...]}
    if isinstance(data, list):
        rows = data
    elif isinstance(data, dict):
        rows = data.get("data", [])
    else:
        rows = []

    rows = rows[:max_records]

    fields = ["symbol", "company", "headline", "ann_date", "attachment", "pdfLink"]
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fields)
        w.writeheader()
        for r in rows:
            symbol = r.get("symbol") or r.get("securitySymbol")
            company = r.get("companyName") or r.get("company")
            headline = r.get("subject") or r.get("headline")
            ann_date = r.get("announcementDate") or r.get("announcedDate")
            attachment_name = r.get("attachmentName") or r.get("fileName")
            attach_url = r.get("attachmentUrl") or r.get("fileUrl")

            w.writerow({
                "symbol": symbol,
                "company": company,
                "headline": headline,
                "ann_date": ann_date,
                "attachment": attachment_name,
                "pdfLink": BASE + attach_url if attach_url and attach_url.startswith("/") else (attach_url or ""),
            })

if __name__ == "__main__":
    fetch_nse_announcements()


ReadTimeout: HTTPSConnectionPool(host='www.nseindia.com', port=443): Read timed out. (read timeout=20)

: 